In [ ]:
import arcpy
import pandas as pd

PROJ_PATH = "C:/Users/bento/final_project/woodpeckerNC/woodpeckerNC.aprx"
proj = arcpy.mp.ArcGISProject(PROJ_PATH)
DB_PATH = proj.defaultGeodatabase 
arcpy.env.workspace = DB_PATH

sensitivity_table = "Pileated_Woodpecker_NC_Sensitivity_Table_1"

field_names = [field.name for field in arcpy.ListFields(sensitivity_table)]
with arcpy.da.SearchCursor(sensitivity_table, 
                           ['CUTOFF', 'FPR', 'TPR', 'FNR', 'TNR', 'SENSE', 'SPEC']) as cursor:
    vals = [
        {
            "cutoff": cutoff,
            "FP": FP,
            "TP": TP,
            "FN": FN,
            "TN": TN,
            "recall": recall,
            "specificity": specificity,
            "precision": TP / (TP + FP),
            "f1": 2 * ((TP / (TP + FP)) * recall) / ((TP / (TP + FP)) + recall),
        }
        for (cutoff, FP, TP, FN, TN, recall, specificity) in cursor
    ]
df = pd.DataFrame.from_dict(vals)

print(f"Max F1: {round(max(df.f1), 4)}")
print(f"Best Cutoff: {round(df.loc[df.f1 == max(df.f1)].cutoff.iloc[0], 2)}")

df


In [6]:
import os
import pickle

model_data_path = "C:/Users/bento/final_project/woodpeckerNC/data/model_data"
s = "Pileated_Woodpecker"
file = os.path.join(model_data_path, f"{s}_model_data.pickle")
print(file)

with open(file, "rb") as f:
    cached_model_data = pickle.load(f)

cached_model_data['params']

C:/Users/bento/final_project/woodpeckerNC/data/model_data\Pileated_Woodpecker_model_data.pickle


{'number_of_iterations': 20,
 'basis_expansion_functions': 'THRESHOLD',
 'relative_weight': 50,
 'number_knots': 50,
 'spatial_thinning': 'THINNING',
 'link_function': 'CLOGLOG',
 'thinning_distance_band': '2500 Meters'}

In [ ]:
log_path = os.path.join(model_data_path, "model_training_logs")

all_params = 
files = os.listdir(log_path)
    for file in files:
        file_path = os.path.join(log_path, file)
        with open(file_path, 'r') as f:
            file_dict = json.load(f)

In [12]:
import itertools

parameter_grid = {
    "number_of_iterations": [10, 20],
    "basis_expansion_functions":["HINGE", "LINEAR", "THRESHOLD", "QUADRATIC"], # "PRODUCT"
    "relative_weight": [35, 50, 100],
    "number_knots":[10, 50], 
    "spatial_thinning": ["NO_THINNING", "THINNING"],
    "link_function": ["CLOGLOG"], # "LOGISTIC"
    "thinning_distance_band": ["1000 Meters", "2500 Meters"] # "5000 Meters"
}
def getAllCombos(parameter_grid:dict) -> dict:
    # Generate all combinations of parameters
    if "NO_THINNING" not in parameter_grid["spatial_thinning"]:
        all_combinations = list(itertools.product(*parameter_grid.values()))
    elif "THINNING" not in parameter_grid["spatial_thinning"]:
        # Generate combinations for NO_THINNING
        all_combinations = [
            (None, basis_expansion, weight, knots, "NO_THINNING", link_function, None)
            for basis_expansion, weight, knots, link_function
            in itertools.product(
                parameter_grid["basis_expansion_functions"],
                parameter_grid["relative_weight"],
                parameter_grid["number_knots"],
                parameter_grid["link_function"]
            )
        ]
    else:
        # Generate combinations for NO_THINNING
        no_thinning_combinations = [
            (None, basis_expansion, weight, knots, "NO_THINNING", link_function, None)
            for basis_expansion, weight, knots, link_function
            in itertools.product(
                parameter_grid["basis_expansion_functions"],
                parameter_grid["relative_weight"],
                parameter_grid["number_knots"],
                parameter_grid["link_function"]
            )
        ]
        # Generate combinations for THINNING
        thinning_combinations = list(itertools.product(
            parameter_grid["number_of_iterations"],
            parameter_grid["basis_expansion_functions"],
            parameter_grid["relative_weight"],
            parameter_grid["number_knots"],
            ["THINNING"],
            parameter_grid["link_function"],
            parameter_grid["thinning_distance_band"]
        ))
        # Combine the two lists of combinations
        all_combinations = no_thinning_combinations + thinning_combinations
    return all_combinations

len(getAllCombos(parameter_grid))

120

In [23]:
import arcpy
import arcpy.mp as mp
import os
import pandas as pd
from birds import Bird
import pickle

with open("C:/Users/bento/final_project/woodpeckerNC/data/species_df.pickle", "rb") as f:
    species_df = pickle.load(f)

project_path="C:/Users/bento/final_project/woodpeckerNC/woodpeckerNC.aprx"
wspace="C:/Users/bento/final_project/woodpeckerNC/woodpeckerNC.gdb"
data_path="C:/Users/bento/final_project/woodpeckerNC/data"
output_folder="maps"

# Set workspace
arcpy.env.workspace = wspace

# Create an output folder for the PDFs
output_folder = os.path.join(data_path, "maps")
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get trained rasters
trained_rasters = arcpy.ListRasters("*_NC_Trained_Raster")

# Get bird/raster name key/value pairs
brd_rasters = [{Bird(species_df, name).formatted_name + \
                "_NC_Trained_Raster":Bird(species_df, name).formatted_name.replace("_", " ")}\
                for name in species_df.species_name.unique() \
                if Bird(species_df, name).formatted_name + "_NC_Trained_Raster" in trained_rasters]
brd_rasters = {key: value for d in brd_rasters for key, value in d.items()}


# Define project
project = arcpy.mp.ArcGISProject(project_path)

In [22]:
# Loop through each trained raster
for raster in brd_rasters.keys():
    # Get species name
    species_name = brd_rasters[raster]
    print(f"Adding {species_name} raster layer to map...")

    # Create a new map and add it to the project
    m = project.listMaps("Map")[0]

    # List layers in map
    lyr_name = raster + "_Lyr"
    layers = [l.name for l in m.listLayers()]
    
    # Hide all existing layers
    if len(m.listLayers()) > 0:
        for l in m.listLayers():
            l.visible = False

    if lyr_name not in layers:
        # Create a raster layer from the raster
        print(f"Adding {lyr_name} to map...")
        raster_layer = arcpy.MakeRasterLayer_management(raster, raster + "_Lyr")
        m.addLayer(raster_layer[0])
    else:
        print(f"{lyr_name} already in map")

    # Define layer, make visible
    layer = [l for l in m.listLayers() if l.name == lyr_name][0]
    layer.visible = True

    print(f"Creating a layout for {species_name}...")
    # Get the default layout
    layout = project.listLayouts("Layout")[0]

    # Update the title of the layout
    title = [el for el in layout.listElements("TEXT_ELEMENT")][0]
    print("Updating title in layout...")
    title.text = species_name + " Modeled Distribution"
    
    # Zoom to the extent of the raster layer
    print("Zooming to map layer extent...")
    mf = layout.listElements("MAPFRAME_ELEMENT")[0]
    mf.zoomToAllLayers()

    # Export the layout to a PDF
    pdf_path = os.path.join(output_folder, raster.replace('Trained_Raster', 'Dist') + ".pdf")
    print(f"Exporting {species_name} layout to {pdf_path}...")
    layout.exportToPDF(pdf_path)
    
    # Hide the added raster layer from the map
    layer.visible = False

# Save the current state of the project
project.save()

Adding Woodpecker, Pileated raster layer to map...
Pileated_Woodpecker_NC_Trained_Raster_Lyr already in map
Creating a layout for Woodpecker, Pileated...
Updating title in layout...
Zooming to map layer extent...
Exporting Woodpecker, Pileated layout to C:/Users/bento/final_project/woodpeckerNC/data\maps\Pileated_Woodpecker_NC_Dist.pdf...
Adding Flicker, Northern (Yellow-shafted) raster layer to map...
Northern_Yellow_shafted_Flicker_NC_Trained_Raster_Lyr already in map
Creating a layout for Flicker, Northern (Yellow-shafted)...
Updating title in layout...
Zooming to map layer extent...
Exporting Flicker, Northern (Yellow-shafted) layout to C:/Users/bento/final_project/woodpeckerNC/data\maps\Northern_Yellow_shafted_Flicker_NC_Dist.pdf...
Adding Woodpecker, Red-headed raster layer to map...
Red_headed_Woodpecker_NC_Trained_Raster_Lyr already in map
Creating a layout for Woodpecker, Red-headed...
Updating title in layout...
Zooming to map layer extent...
Exporting Woodpecker, Red-headed 

OSError: C:/Users/bento/final_project/woodpeckerNC/woodpeckerNC.aprx